## IDS Project
#### This project will create an IDS Detection Machine Learning model, which will predict DoS attacks and flag them as such. 
By Chadi Amzil

In [70]:
!pip install pandas
!pip install seaborn
!pip install pyarrow
!pip install xgboost
!pip install catboost
!pip install flask
!pip install catboost
!pip install hdbscan
!pip install sklearn
!pip install numpy
!pip install matplotlib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import glob
import joblib


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
    

In [71]:
#Supressing warnings generated by the code
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [72]:
# setting pd to display all maximum amount of rows,
# with a scrollable option
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### Extract

In [73]:
#Defining the extract function
def extract() -> pd.DataFrame:
    """This functions combines all the csv and json files into a dataframe

    Args:
        None: The function reads all csv, json files, 
        & parquet files in the working directory
        
    Returns:
        data (pd.DataFrame): All data sources combined in a single dataframe
    """
    ## create an empty dataframe to hold all the data
    # The columns' names are given based on an earlier exploration of a csv file
    data = []
    ## glob.glob('*.csv') returns ['ids_0.csv', 'ids_1.csv', 'ids_2.csv']
    # 
    for csvfile in glob.glob('*.csv'):
        #adding the extracted csv files into the dataframe
        data.append(pd.read_csv(csvfile))

    
    ## glob.glob('*.json') 
    # returns ['ids_3.json', 'ids_4.json', 'ids_7.json', 'ids_9.json', 'ids_10.json]
    for jsonfile in glob.glob('*.json'):
        # adding the extracted json files into the dataframe
        data.append(pd.read_json(jsonfile, lines=True))


    ## glob.glob('*.parquet') 
    # returns ['ids_5.parquet', 'ids_6.parquet', 'ids_8.parquet', 'ids_11.parquet']
    for parquetfile in glob.glob('*.parquet'):
        # adding the extracted json files into the dataframe
        data.append(pd.read_parquet(parquetfile))
    # concatenate the loaded data to the data frame
    data = pd.concat(data, ignore_index=True)
    return data


In [74]:
# Print the documentation of the extract() function
print(extract.__doc__)

This functions combines all the csv and json files into a dataframe

    Args:
        None: The function reads all csv, json files, 
        & parquet files in the working directory

    Returns:
        data (pd.DataFrame): All data sources combined in a single dataframe
    


In [75]:
# Extracting the dataframe
data = extract()
# Printing head to confirm extraction
data.head()


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,87261,1,1,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,443,523,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,53,94304,1,1,56,338,56,56,56.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,53,207,2,2,84,246,42,42,42.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,123,69022032,2,2,96,96,48,48,48.0,0.0,...,32,22276.0,0.0,22276,22276,69000000.0,0.0,69000000,69000000,BENIGN


In [76]:
#Confriming shape of extracting dataframe
data.shape

(61128, 79)

### Transform

In [77]:
# check if any of the columns has a NaN (missing) value
data.isna().sum()

 Destination Port                 0
 Flow Duration                    0
 Total Fwd Packets                0
 Total Backward Packets           0
Total Length of Fwd Packets       0
 Total Length of Bwd Packets      0
 Fwd Packet Length Max            0
 Fwd Packet Length Min            0
 Fwd Packet Length Mean           0
 Fwd Packet Length Std            0
Bwd Packet Length Max             0
 Bwd Packet Length Min            0
 Bwd Packet Length Mean           0
 Bwd Packet Length Std            0
Flow Bytes/s                    111
 Flow Packets/s                  33
 Flow IAT Mean                    0
 Flow IAT Std                     0
 Flow IAT Max                     0
 Flow IAT Min                     0
Fwd IAT Total                     0
 Fwd IAT Mean                     0
 Fwd IAT Std                      0
 Fwd IAT Max                      0
 Fwd IAT Min                      0
Bwd IAT Total                     0
 Bwd IAT Mean                     0
 Bwd IAT Std                

##### Two columns have NaN values, "Flow Bytes/s" & " Flow Packets/s"
##### Also note that some columns have an extra space string at the start, which may cause an errors in the code.

In [78]:
# Exploring data types
data.dtypes

 Destination Port                 int64
 Flow Duration                    int64
 Total Fwd Packets                int64
 Total Backward Packets           int64
Total Length of Fwd Packets       int64
 Total Length of Bwd Packets      int64
 Fwd Packet Length Max            int64
 Fwd Packet Length Min            int64
 Fwd Packet Length Mean         float64
 Fwd Packet Length Std          float64
Bwd Packet Length Max             int64
 Bwd Packet Length Min            int64
 Bwd Packet Length Mean         float64
 Bwd Packet Length Std          float64
Flow Bytes/s                    float64
 Flow Packets/s                 float64
 Flow IAT Mean                  float64
 Flow IAT Std                   float64
 Flow IAT Max                     int64
 Flow IAT Min                     int64
Fwd IAT Total                     int64
 Fwd IAT Mean                   float64
 Fwd IAT Std                    float64
 Fwd IAT Max                      int64
 Fwd IAT Min                      int64


In [79]:
# I will downcast the integers for memory efficiency
data = data.apply(lambda col: pd.to_numeric(col, downcast='integer')
                  if col.dtype =='int64' else col)
data.dtypes

 Destination Port                 int32
 Flow Duration                    int32
 Total Fwd Packets                int16
 Total Backward Packets           int16
Total Length of Fwd Packets       int32
 Total Length of Bwd Packets      int32
 Fwd Packet Length Max            int16
 Fwd Packet Length Min            int16
 Fwd Packet Length Mean         float64
 Fwd Packet Length Std          float64
Bwd Packet Length Max             int16
 Bwd Packet Length Min            int16
 Bwd Packet Length Mean         float64
 Bwd Packet Length Std          float64
Flow Bytes/s                    float64
 Flow Packets/s                 float64
 Flow IAT Mean                  float64
 Flow IAT Std                   float64
 Flow IAT Max                     int32
 Flow IAT Min                     int32
Fwd IAT Total                     int32
 Fwd IAT Mean                   float64
 Fwd IAT Std                    float64
 Fwd IAT Max                      int32
 Fwd IAT Min                      int32


In [80]:
# Checking how many unique values are available
print(data.nunique())

 Destination Port                 886
 Flow Duration                  39517
 Total Fwd Packets                104
 Total Backward Packets           115
Total Length of Fwd Packets      1879
 Total Length of Bwd Packets     1206
 Fwd Packet Length Max            872
 Fwd Packet Length Min            100
 Fwd Packet Length Mean          3875
 Fwd Packet Length Std           7011
Bwd Packet Length Max             666
 Bwd Packet Length Min            251
 Bwd Packet Length Mean          1263
 Bwd Packet Length Std           1288
Flow Bytes/s                    34585
 Flow Packets/s                 42564
 Flow IAT Mean                  33160
 Flow IAT Std                   14954
 Flow IAT Max                   13986
 Flow IAT Min                    4348
Fwd IAT Total                   11221
 Fwd IAT Mean                   13856
 Fwd IAT Std                    11993
 Fwd IAT Max                    11090
 Fwd IAT Min                     4922
Bwd IAT Total                   14833
 Bwd IAT Mea

In [81]:
# Comparing values of two header length table,
# to verify whether it's a duplicate
identical = data[' Fwd Header Length'].equals(data[' Fwd Header Length.1'])
identical

True

In [82]:
def transform(data: pd.DataFrame) -> pd.DataFrame:
     """This function performs the transformations:
    1. Removing duplicates from the extracted data
    2. Handling missing data in "Flow Bytes/s" & "Flow Packets/s
    3. No metric conversion needed, since the data is not metric
      Args:
        data (pd.DataFrame): extracted data ready for transformations

    Returns:
        pd.DataFrame: transformed data
        """
    # Removing duplicates 
     data = data.drop_duplicates()
     # Remove blank spaces in front of colomn names
     data.columns = data.columns.str.strip()
     # Removing the duplicate column "Fwd Header Length.1"
     data.drop(columns='Fwd Header Length.1')
     ## Fill missing values only in these columns with 0, 
     ## Since missing value in flow bytes and flow packets signify no traffic
     data['Flow Bytes/s'] = data['Flow Bytes/s'].fillna(0)
     data['Flow Packets/s'] = data['Flow Packets/s'].fillna(0)

     # return transformed data
     return data


In [83]:
# Print the documentation of the transform() function
print(transform.__doc__)

This function performs the transformations:
    1. Removing duplicates from the extracted data
    2. Handling missing data in "Flow Bytes/s" & "Flow Packets/s
    3. No metric conversion needed, since the data is not metric
      Args:
        data (pd.DataFrame): extracted data ready for transformations

    Returns:
        pd.DataFrame: transformed data
        


In [84]:
#Executing transform function
data = transform(data)


## Load

In [85]:
# Storing the transformed dataset as a CSV file to provide a simple format
# for preprocessing and data manipulation
def loadcsv(data: pd.DataFrame) -> None:
    """This function loads the argument dataframe into a csv file

    Args:
        data (pd.DataFrame): extracted and transformed dataframe
    """
    data.to_csv('ids_transformed.csv', index= False)

In [86]:
# Print the documentation of the loadcsv() function
print(loadcsv.__doc__)

This function loads the argument dataframe into a csv file

    Args:
        data (pd.DataFrame): extracted and transformed dataframe
    


In [87]:
# Executing loadcsv function
data= loadcsv(data)